In [1]:
import sys
import networkx as nx
from node2vec import Node2Vec as n2v
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

In [2]:
"""
ppi.py
This file uses the node2vec algorithm to embed gene nodes and find similar genes. 
Author: Christopher Sun

Usage: python ppi.py diseaseGeneFile interactionNetworkFile
"""

import sys
import networkx as nx
from node2vec import Node2Vec as n2v
import numpy as np

class PPI(object):
    """
    Object to embed gene nodes and calculate their similarity with a list of diseased genes
    """

    # Function to load the disease gene file and the interaction network file
    # Inputs:
    #       diseaseGeneFile: path to disease gene file
    #       interactionNetworkFile: path to interaction network file
    # Returns:
    #       None
    def load_data(self, diseaseGeneFile, interactionNetworkFile):
        self.disease_genes = []
        with open(diseaseGeneFile, "r") as f:
            for line in f:
                self.disease_genes.append(line.strip())
        self.interactions = []
        with open(interactionNetworkFile, "r") as f:
            for line in f:
                contents = line.split()
                self.interactions.append((contents[0], contents[1]))
        G_interactions = nx.Graph()
        G_interactions.add_edges_from(self.interactions)
        self.interactions = G_interactions
    
    # Function to calculate the embedding of each node using node2vec
    # Inputs:
    #       None
    # Returns:
    #       a tuple containing the list of nodes and their corresponding embeddings
    def calculate_embedding(self):
        g_emb = n2v(self.interactions, dimensions=64, walk_length=30, num_walks=100, workers=1, seed=42)
        model = g_emb.fit(window=3, min_count=1, batch_words=4, workers=1, seed=42)
        embeddings = [model.wv.get_vector(str(node)) for node in self.interactions.nodes]
        return list(self.interactions.nodes), embeddings

    # Function to get a set of genes that are close to disease genes based on a threshold
    # Inputs:
    #       gene_nodes: list of nodes
    #       gene_embeddings: list of embeddings corresponding to gene_nodes
    #       threshold: upper threshold to be considered similar
    # Returns:
    #       a set of similar genes
    def get_close_genes(self, gene_nodes, gene_embeddings, threshold):
        similar_genes = set(self.disease_genes)
        disease_gene_embeddings = np.array([gene_embeddings[i] for i in range(len(gene_embeddings)) if gene_nodes[i] in self.disease_genes])
        gene_embeddings_norm = gene_embeddings / np.linalg.norm(gene_embeddings, keepdims=True, axis=1)
        disease_gene_embeddings_norm = disease_gene_embeddings / np.linalg.norm(disease_gene_embeddings, keepdims=True, axis=1)
        distances_all = 1 - np.dot(gene_embeddings_norm, disease_gene_embeddings_norm.T)
        min_distances = np.min(distances_all, axis=1)
        for i in range(len(min_distances)):
            if min_distances[i] <= threshold:
                similar_genes.add(gene_nodes[i])
        return similar_genes

In [3]:
ppi = PPI()
ppi.load_data("disease_gene_list.txt", "interaction_network_filtered.txt")
nodes, embeddings = ppi.calculate_embedding()

Computing transition probabilities:   0%|          | 0/1476 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [01:41<00:00,  1.02s/it]


In [4]:
genes_all_thresholds = []
for threshold in [0.05, 0.1, 0.2, 0.3]:   
    genes_all_thresholds.append(ppi.get_close_genes(nodes, embeddings, threshold))
print([len(i) for i in genes_all_thresholds])

[22, 22, 22, 26]


In [5]:
genes_all_thresholds[1]

{'ADRB2',
 'AKT1',
 'AR',
 'AVP',
 'C3',
 'CASP3',
 'CTNNB1',
 'IL13',
 'IL1B',
 'IL6',
 'KRAS',
 'MAPK1',
 'MAPK14',
 'MAPK3',
 'MMP9',
 'NFKB1',
 'POMC',
 'PTGER1',
 'RELA',
 'TGFB1',
 'TIMP1',
 'TP53'}

In [7]:
genes_all_thresholds[3]

{'ADRB2',
 'AKT1',
 'AR',
 'AVP',
 'C3',
 'CASP3',
 'CTNNB1',
 'GSK3B',
 'IL13',
 'IL1B',
 'IL6',
 'KRAS',
 'MAPK1',
 'MAPK14',
 'MAPK3',
 'MMP9',
 'MYC',
 'NFKB1',
 'POMC',
 'PRKCD',
 'PTGER1',
 'RELA',
 'SRC',
 'TGFB1',
 'TIMP1',
 'TP53'}

In [18]:
not_considered = set()
for i in ppi.interactions.edges():
    if not i[0] in genes_all_thresholds[1] and not i[1] in genes_all_thresholds[1]:
        not_considered.add(i[0])
        not_considered.add(i[1])

In [24]:
len(not_considered)

1454

In [25]:
geneset_data = {}
with open("c2.cp.kegg.v6.2.symbols.filtered.gmt", "r") as f:
    for line in f:
        contents = line.split()
        geneset_data[contents[0]] = contents[2:]

In [33]:
len(ppi.interactions.nodes)

1476

In [42]:
"TPI1" in ppi.interactions.nodes

True

In [26]:
geneset_data

{'KEGG_GLYCOLYSIS_GLUCONEOGENESIS': ['ACSS2',
  'GCK',
  'PGK2',
  'PGK1',
  'PDHB',
  'PDHA1',
  'PDHA2',
  'PGM2',
  'TPI1',
  'ACSS1',
  'FBP1',
  'ADH1B',
  'HK2',
  'ADH1C',
  'HK1',
  'HK3',
  'ADH4',
  'PGAM2',
  'ADH5',
  'PGAM1',
  'ADH1A',
  'ALDOC',
  'ALDH7A1',
  'LDHAL6B',
  'PKLR',
  'LDHAL6A',
  'ENO1',
  'PKM2',
  'PFKP',
  'BPGM',
  'PCK2',
  'PCK1',
  'ALDH1B1',
  'ALDH2',
  'ALDH3A1',
  'AKR1A1',
  'FBP2',
  'PFKM',
  'PFKL',
  'LDHC',
  'GAPDH',
  'ENO3',
  'ENO2',
  'PGAM4',
  'ADH7',
  'ADH6',
  'LDHB',
  'ALDH1A3',
  'ALDH3B1',
  'ALDH3B2',
  'ALDH9A1',
  'ALDH3A2',
  'GALM',
  'ALDOA',
  'DLD',
  'DLAT',
  'ALDOB',
  'G6PC2',
  'LDHA',
  'G6PC',
  'PGM1',
  'GPI'],
 'KEGG_CITRATE_CYCLE_TCA_CYCLE': ['IDH3B',
  'DLST',
  'PCK2',
  'CS',
  'PDHB',
  'PCK1',
  'PDHA1',
  'LOC642502',
  'PDHA2',
  'LOC283398',
  'FH',
  'SDHD',
  'OGDH',
  'SDHB',
  'IDH3A',
  'SDHC',
  'IDH2',
  'IDH1',
  'ACO1',
  'ACLY',
  'MDH2',
  'DLD',
  'MDH1',
  'DLAT',
  'OGDHL',
  'PC',
  

In [20]:
disease_genes, G_interactions = load_data("disease_gene_list.txt", "interaction_network_filtered.txt")

In [21]:
gene_nodes, embeddings = calculate_embedding(G_interactions)

Computing transition probabilities:   0%|          | 0/1476 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [01:40<00:00,  1.01s/it]


In [61]:
def get_close_genes(gene_nodes, gene_embeddings, threshold):
    similar_genes = set(disease_genes)
    disease_gene_embeddings = np.array([gene_embeddings[i] for i in range(len(gene_embeddings)) if gene_nodes[i] in disease_genes])
    norm_gene_embeddings = gene_embeddings / np.linalg.norm(gene_embeddings, keepdims=True)
    print(np.linalg.norm(gene_embeddings, axis=1, keepdims=True).shape)
    print(np.linalg.norm(gene_embeddings, keepdims=True).shape)
    norm_disease_embeddings = disease_gene_embeddings / np.linalg.norm(disease_gene_embeddings, axis=1, keepdims=True)
    cosine_similarities = np.dot(norm_gene_embeddings, norm_disease_embeddings.T)
    cosine_distances = 1 - cosine_similarities
    min_distances = np.min(cosine_distances, axis=1)
    for i, distance in enumerate(min_distances):
        if distance <= threshold:
            similar_genes.add(gene_nodes[i])
    return similar_genes

In [63]:
def get_close_genes(gene_nodes, gene_embeddings, threshold):
    similar_genes = set(disease_genes)
    disease_gene_embeddings = np.array([gene_embeddings[i] for i in range(len(gene_embeddings)) if gene_nodes[i] in disease_genes])
    norm_gene_embeddings = gene_embeddings / np.linalg.norm(gene_embeddings, keepdims=True, axis=1)
    norm_disease_embeddings = disease_gene_embeddings / np.linalg.norm(disease_gene_embeddings, keepdims=True, axis=1)
    distances_all = 1 - np.dot(norm_gene_embeddings, norm_disease_embeddings.T)
    min_distances = np.min(distances_all, axis=1)
    for i in range(len(min_distances)):
        if min_distances[i] <= threshold:
            similar_genes.add(gene_nodes[i])
    return similar_genes

In [65]:
get_close_genes(gene_nodes, embeddings, 0)

{'ADRB2', 'AKT1', 'CTNNB1', 'MAPK1', 'MAPK14', 'MAPK3', 'NFKB1', 'TP53'}

In [51]:
disease_gene_embeddings = [embeddings[i] for i in range(len(embeddings)) if gene_nodes[i] in disease_genes]

In [7]:
min(cosine_distances(embeddings.reshape(1,-1), disease_gene_embeddings)[0])

ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.

In [ ]:
cosine_distances()

In [44]:
get_close_genes(disease_genes, gene_nodes, embeddings, 0.0)

NameError: name 'get_close_genes' is not defined

In [44]:
import pandas as pd

In [59]:
exp_data = pd.read_csv("GSE25628_filtered_expression.txt", sep="\t", index_col=0)

In [60]:
samp_data = {}
with open("GSE25628_samples.txt", "r") as f:
    for line in f:
        contents = line.split()
        samp_data[contents[0]] = contents[1]
geneset_data = {}
with open("c2.cp.kegg.v6.2.symbols.filtered.gmt", "r") as f:
    for line in f:
        contents = line.split()
        geneset_data[contents[0]] = contents[2:]

In [62]:
healthy_idxs = []
sick_idxs = []
for i in range(1, len(exp_data.columns)):
    if samp_data[exp_data.columns[i]] == "1":
        sick_idxs.append(i)
    else:
        healthy_idxs.append(i)

In [70]:
logFC = exp_data.iloc[:,sick_idxs].mean(axis=1) - exp_data.iloc[:,healthy_idxs].mean(axis=1)
gene_rank = logFC.sort_values(ascending=False)

In [73]:
list(gene_rank.index)

['PTGIS',
 'C7',
 'CNN1',
 'ACTG2',
 'TCF21',
 'TAGLN',
 'WISP2',
 'KCNMB1',
 'ACKR1',
 'GPC3',
 'PDLIM3',
 'AQP1',
 'MYH11',
 'CCL14',
 'FZD7',
 'MYL9',
 'TCEAL2',
 'AOC3',
 'FOS',
 'FOSB',
 'DPT',
 'ACTA2',
 'SYNPO',
 'DES',
 'PLA2G2A',
 'KLF2',
 'MT1M',
 'HOXC6',
 'IGFBP6',
 'FMO1',
 'PCP4',
 'MFAP4',
 'FXYD1',
 'ADIRF',
 'AEBP1',
 'MGP',
 'CHRDL1',
 'TPSB2',
 'MN1',
 'FABP4',
 'SCRG1',
 'EGR1',
 'PCOLCE2',
 'BCHE',
 'PLN',
 'PRELP',
 'CCL19',
 'LMOD1',
 'GAS1',
 'KCNK3',
 'ASPN',
 'MMP23A',
 'CLU',
 'NBL1',
 'PLVAP',
 'BGN',
 'PDGFRL',
 'WIF1',
 'HSPB7',
 'MMRN2',
 'ITM2A',
 'PDE2A',
 'MINOS1-NBL1',
 'CFD',
 'THBS2',
 'FRZB',
 'MYLK',
 'DPYSL3',
 'MFAP5',
 'HSPB6',
 'GPX3',
 'CCL21',
 'OLFML1',
 'SELP',
 'RAMP3',
 'LTBP2',
 'PAEP',
 'COMP',
 'FBLN5',
 'CYR61',
 'CPE',
 'EMILIN1',
 'PTRF',
 'SYNM',
 'FEZ1',
 'JAM2',
 'DKK3',
 'C3',
 'CRYAB',
 'CDH3',
 'FXYD6',
 'PTGER3',
 'AHNAK2',
 'SRPX',
 'LIMS2',
 'FMOD',
 'SRPX2',
 'AOX1',
 'CA11',
 'RGS2',
 'TSPAN7',
 'THBS1',
 'COL6A2',
 'SMT